In [1]:
import os
import time
import json

import pandas as pd
import gokart
import luigi
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from mleagueloader import MLEAGUELoader, format_raw_data

In [2]:
DEBUG = True

In [3]:
webdriver.__version__

'4.6.0'

In [4]:
pd.options.display.max_rows = 200

In [5]:
load_dotenv()

MLEAGUE_EMAIL = os.environ['MLEAGUE_EMAIL']
MLEAGUE_PASS=os.environ['MLEAGUE_PASS']

In [ ]:
class LoaderTask(gokart.TaskOnKart):
    season = luigi.IntParameter(default=2022)
    debug = luigi.BoolParameter(default=True)

    def run(self):
        options = Options()

        # Headless
        options.headless = False

        # ブラウザ起動
        driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)
        loader = MLEAGUELoader(driver=driver)

        loader.login_fun(MLEAGUE_EMAIL, MLEAGUE_PASS)

        # 2022シーズンの終了した試合を取得
        schedule = loader.get_match_schedule(self.season)
        schedule.sort(reverse=False)

        if self.debug:
            schedule = schedule[:2]

        df = loader.load(schedule)
        self.dump(df)

In [7]:
df = gokart.build(LoaderTask())

/var/folders/kl/fr_z14214psgvtywhz6p9d5m0000gs/T/ipykernel_79587/763898264.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="chromedriver.exe", options=options)


In [ ]:
game_df, user_df = format_raw_data(df)

In [ ]:
game_df.head(200)

In [ ]:
user_df.head(100)